You might run jupyter notebook through admin: ALT + f + s + a, when you are in folder

In [ ]:
!pip install xlsxwriter

In [ ]:
#Install all packages from python to R for using in Python

import rpy2.robjects.packages as rpackages


utils = importr('utils')
base = importr('base')

utils.chooseCRANmirror()

utils.install_packages('readxl')
utils.install_packages('circular')
utils.install_packages('openxlsx')
utils.install_packages('readr')
utils.install_packages('rlang')

readxl = importr('readxl')
circular = importr('circular')
openxlsx = importr('openxlsx')
readr = importr('readr')
rlang = importr('rlang')


In [95]:
#All packeges needed
import numpy as np
import pandas as pd
import glob
import os
import scipy
import rpy2
from rpy2.robjects.packages import importr, data
from rpy2 import robjects
import math
import xlsxwriter

For all is the same

In [96]:
#RUN!!
#Used when you give path with "/"
def get_name1(file):
    name = str(file)
    replacementStr = ""
    
    n = name.rfind('/')

    name = name.replace(name[0 : n+1], replacementStr, 1)
    name = name.replace(name[-4 :], replacementStr, 1)

    
    return name

In [97]:
#RUN!!
#Used when you give path with "\", normally for windows
def get_name2(file):
    name = str(file)
    replacementStr = ""
    
    n = name.rfind("\\")

    name = name.replace(name[0 : n+1], replacementStr, 1)
    name = name.replace(name[-4 :], replacementStr, 1)

    
    return name

##### Read file

In [98]:
#RUN!!
def read_edit_csv3(file):
    df = pd.read_csv(file, encoding="utf-8", index_col=False)
    name = get_name1(file)

    df["TrackName"] = name
    df = df.filter(['secs into Track','TrackName'])
    df.rename(columns={'secs into Track': name}, inplace=True)
    df = df.drop('TrackName', axis = 1)

    return df

# 80 - Functions on one sheet

### IOI limits calculation

##### Output just ioi data - for concatenate in IOI

In [99]:
#RUN
def ioi_lim_alone_80(file):
    df = read_edit_csv3(file)
    name = get_name1(file)
    
    df['Difference'] = df.diff()
    first = df.iloc[:,0].tolist()
    second = df.iloc[:,1].tolist()

    for vrstica1 in first:
        for i, vrstica2 in enumerate(second):
            if vrstica2 == vrstica1 or vrstica2 < 0.1:
                vrstica2 = float('NaN')
                second[i] = vrstica2
            
    df1 = pd.DataFrame({'ioi':second})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)
    df = df.drop('Difference', axis = 1)
    df = df.rename(columns={"ioi": name})
    df = df.iloc[: , 1:]
    return df

Add to raw data clean (2.625<x<21)

In [100]:
#RUN!
def ioi_lim_clean_80(file):
    df = read_edit_csv3(file)
    
    first = df.iloc[:,0].tolist()
    for i, line in enumerate(first):
        if line < 2.625 or line > 21:
            first[i] = 0
    df1 = pd.DataFrame({'clean': first})
    
    frames = [df,df1]
    df = pd.concat(frames, axis=1)

    return df

#### Outlier filtering 

##### Delete first 10 valid taps

After filtering, the first 10 valid taps are deleted from the beginning of each trial)

In [101]:
#RUN!
def ioi_lim_filt10_80(file):
    df = ioi_lim_clean_80(file)
    
    df['diff'] = df['clean'].diff()
    first = df.iloc[:,1].tolist()
    second = df.iloc[:,2].tolist()
    
    delete = 0
    for vrstica1 in first:
        for i, vrstica2 in enumerate(second):
            if vrstica2 == vrstica1 or vrstica2 < 0.1:
                second[i] = 0
            elif vrstica2 != 0 and delete < 10:
                second[i] = 0
                delete += 1
    
                           
    df1 = pd.DataFrame({'filter10':second})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)

    return df

Function for clean, filter and delete first 10 valid taps

In [102]:
#RUN!!
def ioi_lim_clean10_80(file):
    df = ioi_lim_filt10_80(file)
       
    
    df['filtered'] = np.where(df['filter10'] == 0,
                             df['filter10'], df['clean'])
    
    df['filtered'].replace(to_replace = 0, value = float('NaN'), inplace=True)

    return df #return data frame with columns: input track, clean, diff, filter10, filtered

### Calculations

Add REF column to the data frame

In [103]:
#RUN!!
def df_prep_80(file):
    df = ioi_lim_clean10_80(file)
    df = df.drop(['clean', 'diff', 'filter10'], axis=1)
    
    s = [0]
    x = 0
    while x < 22:
        x += 0.75
        s.append(x)

    df['REF'] = pd.Series(s)

    return df

Diff, nearest values

In [104]:
#RUN!!
def df_diff_angle_80(file):
    df = df_prep_80(file)
    
    df['diff'] = (pd.merge_asof(df['filtered'].dropna().sort_values().reset_index(),
                                df[['REF']].dropna().sort_values('REF'),
                                left_on='filtered', right_on='REF', direction='nearest')
                    .set_index('index')['REF'].rsub(df['filtered'])
                  )
    
    df['angle'] = (df['diff']/0.75)*360
    
    return df

### IOI data

Give you data frame with ioi data from all participants.

In [105]:
#RUN!
def get_files_80(directory):
    seznam = []

    for dirpath, dirnames, filenames in os.walk(directory): #directory with all folders of participants
        for filename in [f for f in filenames if f.endswith("80.csv")]: #search for all 80 files
            seznam.append(os.path.join(dirpath, filename))
            
    return seznam

In [106]:
#RUN!!
def ioi_frame_80(directory):
    
    seznam = get_files_80(directory)
    li = []    
    
    for filename in seznam:
        df = ioi_lim_alone_80(filename)
        li.append(df)

    frame = pd.concat(li, axis=1, ignore_index=False)
    
    return frame

##### Concatenate just IOI data

### IOI-limits

Add columns with all values together and calculate M, SD, Q1, Q3, IQR, MAX, MIN.

In [107]:
#RUN!
def list_ioi_calc_80(directory):
    frame = ioi_frame_80(directory)
    frame['combined']= frame.values.tolist()
    li = []

    for row in frame['combined']:
        for vsak in row:
            li.append(vsak)

    new_array = [x for x in li if math.isnan(x) == False]
    se = pd.Series(new_array)
    
    df_ioi = pd.DataFrame([new_array])
    df_ioi = df_ioi.T
    df_ioi['all_80'] = pd.DataFrame(data=se)
    df_ioi = df_ioi.iloc[: , 1:]
    
    return df_ioi

In [108]:
#RUN!!
def ioi_limits_80(directory):
    frame = ioi_frame_80(directory)
    df_ioi = list_ioi_calc_80(directory)
    
    df_ioi['mean'] = [(df_ioi['all_80'].mean())]+['']*(len(df_ioi)-1)
    frame['mean'] = df_ioi['mean']
    
    df_ioi['std'] = [(df_ioi['all_80'].std())]+['']*(len(df_ioi)-1)
    frame['std'] = df_ioi['std']
    
    df_ioi['q1'] = [(df_ioi['all_80'].quantile(.25, interpolation="nearest"))]+['']*(len(df_ioi)-1)
    frame['q1'] = df_ioi['q1']
    
    df_ioi['q3'] = [(df_ioi['all_80'].quantile(.75, interpolation="nearest"))]+['']*(len(df_ioi)-1)
    frame['q3'] = df_ioi['q3']
    
    df_ioi['IQR'] = [(df_ioi['q3'].iloc[0] - df_ioi['q1'].iloc[0])]+['']*(len(df_ioi)-1)
    frame['IQR'] = df_ioi['IQR']
    
    df_ioi['min'] = [(df_ioi['q1'].iloc[0] - (3*(df_ioi['IQR'].iloc[0])))]+['']*(len(df_ioi)-1)
    frame['min'] = df_ioi['min']
    
    df_ioi['max'] = [(df_ioi['q3'].iloc[0] + (3*(df_ioi['IQR'].iloc[0])))]+['']*(len(df_ioi)-1)
    frame['max'] = df_ioi['max']
    
    return df_ioi.to_excel('df_ioi_80.xlsx'), frame.to_excel('IOI_filter_80.xlsx')

### Circular analysis

In [109]:
#RUN!!
def circular_80(file):
    prep = prep_for_r_80(file)
    circ_in_r_80(prep)

Create xlsx with angel column to send in R

In [110]:
#RUN!!
def prep_for_r_80(file):
    df = df_diff_angle_80(file)
    name = get_name1(file)

    df_a = df.filter(['angle'])
        
    return df_a.to_excel('angle_80.xlsx')

Call R function and provide calculations, merge them in participant dataframe

In [111]:
#RUN!!

#Excecute R circular analysis and make xlsx file with measurement

def circ_in_r_80(prep):

    robjects.r('''
        circ_anal <- function (xlsx_file) {
          library(readxl)
          library(circular)
          library(openxlsx)
          library(readr)
          library(rlang)

          angle_80 <- read_excel(xlsx_file)

          angle_80[1] <- NULL

          circ80 <- data.frame(ID = colnames(angle_80), 
                               RAY = NA,
                               RAYsig = NA,
                               MEAN = NA,
                               Rho = NA)

          for(i in 1:length(circ80$ID)){
            a <- angle_80[,i]
            a_circ <- circular(a, units = "degrees", zero = circular(pi/2), rotation = "clock")
            circ80$RAY[i] <- rayleigh.test(a_circ)$statistic
            circ80$RAYsig[i] <- rayleigh.test(a_circ)$p.value
            circ80$MEAN[i] <- mean.circular(a_circ, na.rm = TRUE)
            circ80$Rho[i] <- rho.circular(a_circ, na.rm = TRUE)

          }


          write.xlsx(circ80,"C:/Users/Neza/Downloads/tapping/circ80_stat.xlsx")

        }

        circ_anal("C:/Users/Neza/Downloads/tapping/angle_80.xlsx")
    ''')

Read excel with circular data and delete angle_80 and circ80_stat files.

In [112]:
#RUN!!!

def final_circ_80(file):
    circular_80(file)
    
    df1 = df_diff_angle_80(file)
    df2 = pd.read_excel('circ80_stat.xlsx')

    df = pd.concat([df1, df2], axis = 1)
    df = df.drop('ID', axis=1)

    os.remove("angle_80.xlsx")
    os.remove("circ80_stat.xlsx")
    
    return df

## Cont MTC

Add abs(DIF) column, calculated both means, add contMTC column.

In [113]:
#RUN!!
def cont_mtc_80(file):
    df = final_circ_80(file)
    
    df['abs_diff'] = df['diff'].abs()
    df['m_diff'] = [np.mean(df['diff'])]+['']*(len(df)-1)
    df['m_abs'] = [np.mean(df['abs_diff'])]+['']*(len(df)-1)
    
    first = df.iloc[:,0].tolist()
    for i, line in enumerate(first):
        if line <= 21:
            first[i] = 0
    df1 = pd.DataFrame({'contMTC': first})

    frames = [df,df1]
    df = pd.concat(frames, axis=1)
    
    return df

### Add ioi filter and proceed last calculations

Concatenate df_ioi_80 excel with cont_mtc_80.

In [114]:
#RUN!!
def final_concat_80(file, directory):
    excels = ioi_limits_80(directory)
    
    df1 = cont_mtc_80(file)
    df2 = pd.read_excel('df_ioi_80.xlsx', usecols=lambda x: 'Unnamed' not in x,)
    
    df2 = df2.drop('all_80', axis=1)
    
    frames = [df1, df2]
    df = pd.concat(frames, axis=1)
    
    os.remove("df_ioi_80.xlsx")
    
    return df

## Cont IOI measures

From contMTC delete if difference is out of calculated ioi limits.

In [115]:
#RUN!!
def pre_ioi_filt_80(file, directory):
    df = final_concat_80(file, directory)
    
    df['Difference'] = df['contMTC'].diff()
    first = df['Difference'].tolist()
    
    for i, vrstica in enumerate(first):
        if vrstica < (df['min'].iloc[0]) or vrstica > (df['max'].iloc[0]):
            first[i] = 0            
            
    df1 = pd.DataFrame({'preIOI': first})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)
    df = df.drop('Difference', axis = 1)
    
    col = df['preIOI'].tolist()
    st = 0

    for i, line in enumerate(col):
        if line != 0 and st <= 29:
            st += 1
        elif line != 0 and st > 29:
            col[i] = 0

    df1 = pd.DataFrame({'preIOI_29': col})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)
    
    return df

ContIOI calculated: delete smaller than 0.1 and outliers based on quartiles of participant (preIOI quartiles).

In [116]:
#RUN!!!

def cont_ioi_final_80(file, directory):
    df = pre_ioi_filt_80(file, directory)
    df['contMTC'].replace(to_replace = 0, value = float('NaN'), inplace=True)
    df['preIOI'].replace(to_replace = 0, value = float('NaN'), inplace=True)
    df['preIOI_29'].replace(to_replace = 0, value = float('NaN'), inplace=True)
    
    first = df['preIOI_29'].tolist()
    
    q1 = np.quantile(first, .25)
    q3 = np.quantile(first, .75)
    iqr = q3 - q1
    
    for i, vrstica in enumerate(first):
        if vrstica < 0.1 or vrstica < (q1 - (3*iqr)) or vrstica > (q3 + (3*iqr)):
            first[i] = 0
            
            
    df1 = pd.DataFrame({'contIOI': first})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)
    df['contIOI'].replace(to_replace = 0, value = float('NaN'), inplace=True)
    
    return df

Last measures: avg, std, IOI elemszám, TÖRÖLNI.

In [117]:
#RUN!!
#Change get_name in final version!

def all_calc_80(file, directory):
    df = cont_ioi_final_80(file, directory)
    name = get_name1(file)
    
    df['mIOIcont'] = [df['contIOI'].mean()]+['']*(len(df)-1)
    df['CVcont'] = [(df['contIOI'].std())/(df['mIOIcont'].iloc[0])]+['']*(len(df)-1)
    
    df['IOIelemszam'] = [((df['contIOI'] != 0).sum())]+['']*(len(df)-1)
    #df['torolni'] = [(df['IOIelemszam'].iloc[0]) - 29]+['']*(len(df)-1)
    
    #df.to_excel(name+".xlsx",
     #        sheet_name='80') #later put in the function with all dataframes, all tempos
    
    return df

# 80m - Functions on one sheet

Everything same as in 80, just on metronome, all calculations are same.

### IOI limits calculation

##### Output just ioi data - for concatenate in IOI

In [118]:
#RUN
def ioi_lim_alone_80m(file):
    df = read_edit_csv3(file)
    name = get_name2(file)
    
    df['Difference'] = df.diff()
    first = df.iloc[:,0].tolist()
    second = df.iloc[:,1].tolist()

    for vrstica1 in first:
        for i, vrstica2 in enumerate(second):
            if vrstica2 == vrstica1 or vrstica2 < 0.1:
                vrstica2 = float('NaN')
                second[i] = vrstica2
            
    df1 = pd.DataFrame({'ioi':second})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)
    df = df.drop('Difference', axis = 1)
    df = df.rename(columns={"ioi": name})
    df = df.iloc[: , 1:]
    return df

Add to raw data clean (2.625<x<21)

In [119]:
#RUN!
def ioi_lim_clean_80m(file):
    df = read_edit_csv3(file)
    #name = get_name(file)
    
    first = df.iloc[:,0].tolist()
    for i, line in enumerate(first):
        if line < 2.625 or line > 21:
            first[i] = 0
    df1 = pd.DataFrame({'clean': first})
    
    frames = [df,df1]
    df = pd.concat(frames, axis=1)

    return df

#### Outlier filtering 

##### Delete first 10 valid taps

After filtering, the first 10 valid taps are deleted from the beginning of each trial)

In [120]:
#RUN!
def ioi_lim_filt10_80m(file):
    df = ioi_lim_clean_80m(file)
    #name = get_name(file)
    
    df['diff'] = df['clean'].diff()
    first = df.iloc[:,1].tolist()
    second = df.iloc[:,2].tolist()
    
    delete = 0
    for vrstica1 in first:
        for i, vrstica2 in enumerate(second):
            if vrstica2 == vrstica1 or vrstica2 < 0.1:
                second[i] = 0
            elif vrstica2 != 0 and delete < 10:
                second[i] = 0
                delete += 1
    
                           
    df1 = pd.DataFrame({'filter10':second})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)

    return df

Function for clean, filter and delete first 10 valid taps

In [121]:
#RUN!!
def ioi_lim_clean10_80m(file):
    df = ioi_lim_filt10_80m(file)
       
    
    df['filtered'] = np.where(df['filter10'] == 0,
                             df['filter10'], df['clean'])
    
    df['filtered'].replace(to_replace = 0, value = float('NaN'), inplace=True)

    return df #return data frame with columns: input track, clean, diff, filter10, filtered

### Calculations

Add REF column to the data frame

In [122]:
#RUN!!
def df_prep_80m(file):
    df = ioi_lim_clean10_80m(file)
    df = df.drop(['clean', 'diff', 'filter10'], axis=1)
    
    s = [0]
    x = 0
    while x < 22:
        x += 0.75
        s.append(x)

    df['REF'] = pd.Series(s)

    return df

Diff, nearest values

In [123]:
#RUN!!
def df_diff_angle_80m(file):
    df = df_prep_80m(file)
    
    df['diff'] = (pd.merge_asof(df['filtered'].dropna().sort_values().reset_index(),
                                df[['REF']].dropna().sort_values('REF'),
                                left_on='filtered', right_on='REF', direction='nearest')
                    .set_index('index')['REF'].rsub(df['filtered'])
                  )
    
    df['angle'] = (df['diff']/0.75)*360
    
    return df

### IOI data

Give you data frame with ioi data from all participants.

In [124]:
#RUN!
def get_files_80m(directory):
    seznam = []

    for dirpath, dirnames, filenames in os.walk(directory): #directory with all folders of participants
        for filename in [f for f in filenames if f.endswith("80m.csv")]: #search for all 80m files
            seznam.append(os.path.join(dirpath, filename))
            
    return seznam

In [125]:
#RUN!!
def ioi_frame_80m(directory):
    
    seznam = get_files_80m(directory)
    li = []    
    
    for filename in seznam:
        df = ioi_lim_alone_80m(filename)
        li.append(df)

    frame = pd.concat(li, axis=1, ignore_index=False)
    
    return frame

##### Concatenate just IOI data

### IOI-limits

Add columns with all values together and calculate M, SD, Q1, Q3, IQR, MAX, MIN.

In [126]:
#RUN!
def list_ioi_calc_80m(directory):
    frame = ioi_frame_80m(directory)
    frame['combined']= frame.values.tolist()
    li = []

    for row in frame['combined']:
        for vsak in row:
            li.append(vsak)

    new_array = [x for x in li if math.isnan(x) == False]
    se = pd.Series(new_array)
    
    df_ioi = pd.DataFrame([new_array])
    df_ioi = df_ioi.T
    df_ioi['all_80m'] = pd.DataFrame(data=se)
    df_ioi = df_ioi.iloc[: , 1:]
    
    return df_ioi

In [127]:
#RUN!!
def ioi_limits_80m(directory):
    frame = ioi_frame_80m(directory)
    df_ioi = list_ioi_calc_80m(directory)
    
    df_ioi['mean'] = [(df_ioi['all_80m'].mean())]+['']*(len(df_ioi)-1)
    frame['mean'] = df_ioi['mean']
    
    df_ioi['std'] = [(df_ioi['all_80m'].std())]+['']*(len(df_ioi)-1)
    frame['std'] = df_ioi['std']
    
    df_ioi['q1'] = [(df_ioi['all_80m'].quantile(.25, interpolation="nearest"))]+['']*(len(df_ioi)-1)
    frame['q1'] = df_ioi['q1']
    
    df_ioi['q3'] = [(df_ioi['all_80m'].quantile(.75, interpolation="nearest"))]+['']*(len(df_ioi)-1)
    frame['q3'] = df_ioi['q3']
    
    df_ioi['IQR'] = [(df_ioi['q3'].iloc[0] - df_ioi['q1'].iloc[0])]+['']*(len(df_ioi)-1)
    frame['IQR'] = df_ioi['IQR']
    
    df_ioi['min'] = [(df_ioi['q1'].iloc[0] - (3*(df_ioi['IQR'].iloc[0])))]+['']*(len(df_ioi)-1)
    frame['min'] = df_ioi['min']
    
    df_ioi['max'] = [(df_ioi['q3'].iloc[0] + (3*(df_ioi['IQR'].iloc[0])))]+['']*(len(df_ioi)-1)
    frame['max'] = df_ioi['max']
    
    return df_ioi.to_excel('df_ioi_80m.xlsx'), frame.to_excel('IOI_filter_80m.xlsx')

### Circular analysis

In [128]:
#RUN!!
def circular_80m(file):
    prep = prep_for_r_80m(file)
    circ_in_r_80m(prep)

Create xlsx with angel column to send in R

In [129]:
#RUN!!
def prep_for_r_80m(file):
    df = df_diff_angle_80m(file)
    name = get_name1(file)

    df_a = df.filter(['angle'])
        
    return df_a.to_excel('angle_80m.xlsx')

Call R function and provide calculations, merge them in participant dataframe

In [130]:
#RUN!!

#Excecute R circular analysis and make xlsx file with measurement

def circ_in_r_80m(prep):

    robjects.r('''
        circ_anal <- function (xlsx_file) {
          library(readxl)
          library(circular)
          library(openxlsx)
          library(readr)
          library(rlang)

          angle_80m <- read_excel(xlsx_file)

          angle_80m[1] <- NULL

          circ80m <- data.frame(ID = colnames(angle_80m), 
                               RAY = NA,
                               RAYsig = NA,
                               MEAN = NA,
                               Rho = NA)

          for(i in 1:length(circ80m$ID)){
            a <- angle_80m[,i]
            a_circ <- circular(a, units = "degrees", zero = circular(pi/2), rotation = "clock")
            circ80m$RAY[i] <- rayleigh.test(a_circ)$statistic
            circ80m$RAYsig[i] <- rayleigh.test(a_circ)$p.value
            circ80m$MEAN[i] <- mean.circular(a_circ, na.rm = TRUE)
            circ80m$Rho[i] <- rho.circular(a_circ, na.rm = TRUE)

          }


          write.xlsx(circ80m,"C:/Users/Neza/Downloads/tapping/circ80m_stat.xlsx")

        }

        circ_anal("C:/Users/Neza/Downloads/tapping/angle_80m.xlsx")
    ''')

Read excel with circular data and delete angle_80 and circ80_stat files.

In [131]:
#RUN!!!

def final_circ_80m(file):
    circular_80m(file)
    
    df1 = df_diff_angle_80m(file)
    df2 = pd.read_excel('circ80m_stat.xlsx')

    df = pd.concat([df1, df2], axis = 1)
    df = df.drop('ID', axis=1)

    os.remove("angle_80m.xlsx")
    os.remove("circ80m_stat.xlsx")
    
    return df

## Cont MTC

Add abs(DIF) column, calculated both means, add contMTC column.

In [132]:
#RUN!!
def cont_mtc_80m(file):
    df = final_circ_80m(file)
    
    df['abs_diff'] = df['diff'].abs()
    df['m_diff'] = [np.mean(df['diff'])]+['']*(len(df)-1)
    df['m_abs'] = [np.mean(df['abs_diff'])]+['']*(len(df)-1)
    
    first = df.iloc[:,0].tolist()
    for i, line in enumerate(first):
        if line <= 21:
            first[i] = 0
    df1 = pd.DataFrame({'contMTC': first})

    frames = [df,df1]
    df = pd.concat(frames, axis=1)
    
    return df

### Add ioi filter and proceed last calculations

Concatenate df_ioi_80 excel with cont_mtc_80.

In [133]:
#RUN!!
def final_concat_80m(file, directory):
    excels = ioi_limits_80m(directory)
    
    df1 = cont_mtc_80m(file)
    df2 = pd.read_excel('df_ioi_80m.xlsx', usecols=lambda x: 'Unnamed' not in x,)
    
    df2 = df2.drop('all_80m', axis=1)
    
    frames = [df1, df2]
    df = pd.concat(frames, axis=1)
    
    os.remove("df_ioi_80m.xlsx")
    
    return df

## Cont IOI measures

From contMTC delete if difference is out of calculated ioi limits.

In [134]:
#RUN!!
def pre_ioi_filt_80m(file, directory):
    df = final_concat_80m(file, directory)
    
    df['Difference'] = df['contMTC'].diff()
    first = df['Difference'].tolist()
    
    for i, vrstica in enumerate(first):
        if vrstica < (df['min'].iloc[0]) or vrstica > (df['max'].iloc[0]):
            first[i] = 0            
            
    df1 = pd.DataFrame({'preIOI': first})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)
    df = df.drop('Difference', axis = 1)
    
    col = df['preIOI'].tolist()
    st = 0

    for i, line in enumerate(col):
        if line != 0 and st <= 29:
            st += 1
        elif line != 0 and st > 29:
            col[i] = 0

    df1 = pd.DataFrame({'preIOI_29': col})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)
    
    return df

ContIOI calculated: delete smaller than 0.1 and outliers based on quartiles of participant (preIOI quartiles).

In [135]:
#RUN!!!

def cont_ioi_final_80m(file, directory):
    df = pre_ioi_filt_80m(file, directory)
    df['contMTC'].replace(to_replace = 0, value = float('NaN'), inplace=True)
    df['preIOI'].replace(to_replace = 0, value = float('NaN'), inplace=True)
    df['preIOI_29'].replace(to_replace = 0, value = float('NaN'), inplace=True)
    
    first = df['preIOI_29'].tolist()
    
    q1 = np.quantile(first, .25)
    q3 = np.quantile(first, .75)
    iqr = q3 - q1
    
    for i, vrstica in enumerate(first):
        if vrstica < 0.1 or vrstica < (q1 - (3*iqr)) or vrstica > (q3 + (3*iqr)):
            first[i] = 0
            
            
    df1 = pd.DataFrame({'contIOI': first})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)
    df['contIOI'].replace(to_replace = 0, value = float('NaN'), inplace=True)
    
    return df

Last measures: avg, std, IOI elemszám, TÖRÖLNI.

In [136]:
#RUN!!
#Change get_name in final version!

def all_calc_80m(file, directory):
    df = cont_ioi_final_80m(file, directory)
    name = get_name1(file)
    
    df['mIOIcont'] = [df['contIOI'].mean()]+['']*(len(df)-1)
    df['CVcont'] = [(df['contIOI'].std())/(df['mIOIcont'].iloc[0])]+['']*(len(df)-1)
    
    df['IOIelemszam'] = [((df['contIOI'] != 0).sum())]+['']*(len(df)-1)
    #df['torolni'] = [(df['IOIelemszam'].iloc[0]) - 29]+['']*(len(df)-1)
    
    #df.to_excel(name+".xlsx",
     #        sheet_name='80m') #later put in the function with all dataframes, all tempos
    
    return df

# 120

### IOI limits calculation

##### Output just ioi data - for concatenate in IOI

In [137]:
#RUN
def ioi_lim_alone_120(file):
    df = read_edit_csv3(file)
    name = get_name1(file)
    
    df['Difference'] = df.diff()
    first = df.iloc[:,0].tolist()
    second = df.iloc[:,1].tolist()

    for vrstica1 in first:
        for i, vrstica2 in enumerate(second):
            if vrstica2 == vrstica1 or vrstica2 < 0.1:
                vrstica2 = float('NaN')
                second[i] = vrstica2
            
    df1 = pd.DataFrame({'ioi':second})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)
    df = df.drop('Difference', axis = 1)
    df = df.rename(columns={"ioi": name})
    df = df.iloc[: , 1:]
    return df

Add to raw data clean (1.75<x<28)

In [138]:
#RUN!
def ioi_lim_clean_120(file):
    df = read_edit_csv3(file)
    
    first = df.iloc[:,0].tolist()
    for i, line in enumerate(first):
        if line < 1.75 or line > 28:
            first[i] = 0
    df1 = pd.DataFrame({'clean': first})
    
    frames = [df,df1]
    df = pd.concat(frames, axis=1)

    return df

#### Outlier filtering 

##### Delete first 10 valid taps

After filtering, the first 10 valid taps are deleted from the beginning of each trial)

In [139]:
#RUN!
def ioi_lim_filt10_120(file):
    df = ioi_lim_clean_120(file)
    
    df['diff'] = df['clean'].diff()
    first = df.iloc[:,1].tolist()
    second = df.iloc[:,2].tolist()
    
    delete = 0
    for vrstica1 in first:
        for i, vrstica2 in enumerate(second):
            if vrstica2 == vrstica1 or vrstica2 < 0.1:
                second[i] = 0
            elif vrstica2 != 0 and delete < 10:
                second[i] = 0
                delete += 1
    
                           
    df1 = pd.DataFrame({'filter10':second})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)

    return df

Function for clean, filter and delete first 10 valid taps

In [140]:
#RUN!!
def ioi_lim_clean10_120(file):
    df = ioi_lim_filt10_120(file)
       
    
    df['filtered'] = np.where(df['filter10'] == 0,
                             df['filter10'], df['clean'])
    
    df['filtered'].replace(to_replace = 0, value = float('NaN'), inplace=True)

    return df #return data frame with columns: input track, clean, diff, filter10, filtered

### Calculations

Add REF column to the data frame

In [181]:
#RUN!!
def df_prep_120(file):
    df = ioi_lim_clean10_120(file)
    df = df.drop(['clean', 'diff', 'filter10'], axis=1)
    
    s = [0]
    x = 0
    while x < 28:
        x += 0.5
        s.append(x)

    df['REF'] = pd.Series(s)

    return df

Diff, nearest values

In [142]:
#RUN!!
def df_diff_angle_120(file):
    df = df_prep_120(file)
    
    df['diff'] = (pd.merge_asof(df['filtered'].dropna().sort_values().reset_index(),
                                df[['REF']].dropna().sort_values('REF'),
                                left_on='filtered', right_on='REF', direction='nearest')
                    .set_index('index')['REF'].rsub(df['filtered'])
                  )
    
    df['angle'] = (df['diff']/0.5)*360
    
    return df

### IOI data

Give you data frame with ioi data from all participants.

In [143]:
#RUN!
def get_files_120(directory):
    seznam = []

    for dirpath, dirnames, filenames in os.walk(directory): #directory with all folders of participants
        for filename in [f for f in filenames if f.endswith("120.csv")]: #search for all 80 files
            seznam.append(os.path.join(dirpath, filename))
            
    return seznam

In [144]:
#RUN!!
def ioi_frame_120(directory):
    
    seznam = get_files_120(directory)
    li = []    
    
    for filename in seznam:
        df = ioi_lim_alone_120(filename)
        li.append(df)

    frame = pd.concat(li, axis=1, ignore_index=False)
    
    return frame

##### Concatenate just IOI data

### IOI-limits

Add columns with all values together and calculate M, SD, Q1, Q3, IQR, MAX, MIN.

In [145]:
#RUN!
def list_ioi_calc_120(directory):
    frame = ioi_frame_120(directory)
    frame['combined']= frame.values.tolist()
    li = []

    for row in frame['combined']:
        for vsak in row:
            li.append(vsak)

    new_array = [x for x in li if math.isnan(x) == False]
    se = pd.Series(new_array)
    
    df_ioi = pd.DataFrame([new_array])
    df_ioi = df_ioi.T
    df_ioi['all_120'] = pd.DataFrame(data=se)
    df_ioi = df_ioi.iloc[: , 1:]
    
    return df_ioi

In [146]:
#RUN!!
def ioi_limits_120(directory):
    frame = ioi_frame_120(directory)
    df_ioi = list_ioi_calc_120(directory)
    
    df_ioi['mean'] = [(df_ioi['all_120'].mean())]+['']*(len(df_ioi)-1)
    frame['mean'] = df_ioi['mean']
    
    df_ioi['std'] = [(df_ioi['all_120'].std())]+['']*(len(df_ioi)-1)
    frame['std'] = df_ioi['std']
    
    df_ioi['q1'] = [(df_ioi['all_120'].quantile(.25, interpolation="nearest"))]+['']*(len(df_ioi)-1)
    frame['q1'] = df_ioi['q1']
    
    df_ioi['q3'] = [(df_ioi['all_120'].quantile(.75, interpolation="nearest"))]+['']*(len(df_ioi)-1)
    frame['q3'] = df_ioi['q3']
    
    df_ioi['IQR'] = [(df_ioi['q3'].iloc[0] - df_ioi['q1'].iloc[0])]+['']*(len(df_ioi)-1)
    frame['IQR'] = df_ioi['IQR']
    
    df_ioi['min'] = [(df_ioi['q1'].iloc[0] - (3*(df_ioi['IQR'].iloc[0])))]+['']*(len(df_ioi)-1)
    frame['min'] = df_ioi['min']
    
    df_ioi['max'] = [(df_ioi['q3'].iloc[0] + (3*(df_ioi['IQR'].iloc[0])))]+['']*(len(df_ioi)-1)
    frame['max'] = df_ioi['max']
    
    return df_ioi.to_excel('df_ioi_120.xlsx'), frame.to_excel('IOI_filter_120.xlsx')

### Circular analysis

In [147]:
#RUN!!
def circular_120(file):
    prep = prep_for_r_120(file)
    circ_in_r_120(prep)

Create xlsx with angel column to send in R

In [148]:
#RUN!!
def prep_for_r_120(file):
    df = df_diff_angle_120(file)
    name = get_name1(file)

    df_a = df.filter(['angle'])
        
    return df_a.to_excel('angle_120.xlsx')

Call R function and provide calculations, merge them in participant dataframe

In [149]:
#RUN!!

#Excecute R circular analysis and make xlsx file with measurement
#Change to right path, where should be excel files made

def circ_in_r_120(prep):

    robjects.r('''
        circ_anal <- function (xlsx_file) {
          library(readxl)
          library(circular)
          library(openxlsx)
          library(readr)
          library(rlang)

          angle_120 <- read_excel(xlsx_file)

          angle_120[1] <- NULL

          circ120 <- data.frame(ID = colnames(angle_120), 
                               RAY = NA,
                               RAYsig = NA,
                               MEAN = NA,
                               Rho = NA)

          for(i in 1:length(circ120$ID)){
            a <- angle_120[,i]
            a_circ <- circular(a, units = "degrees", zero = circular(pi/2), rotation = "clock")
            circ120$RAY[i] <- rayleigh.test(a_circ)$statistic
            circ120$RAYsig[i] <- rayleigh.test(a_circ)$p.value
            circ120$MEAN[i] <- mean.circular(a_circ, na.rm = TRUE)
            circ120$Rho[i] <- rho.circular(a_circ, na.rm = TRUE)

          }


          write.xlsx(circ120,"C:/Users/Neza/Downloads/tapping/circ120_stat.xlsx")

        }

        circ_anal("C:/Users/Neza/Downloads/tapping/angle_120.xlsx")
    ''')

Read excel with circular data and delete angle_80 and circ80_stat files.

In [150]:
#RUN!!!

def final_circ_120(file):
    circular_120(file)
    
    df1 = df_diff_angle_120(file)
    df2 = pd.read_excel('circ120_stat.xlsx')

    df = pd.concat([df1, df2], axis = 1)
    df = df.drop('ID', axis=1)

    os.remove("angle_120.xlsx")
    os.remove("circ120_stat.xlsx")
    
    return df

## Cont MTC

Add abs(DIF) column, calculated both means, add contMTC column.

In [151]:
#RUN!!
def cont_mtc_120(file):
    df = final_circ_120(file)
    
    df['abs_diff'] = df['diff'].abs()
    df['m_diff'] = [np.mean(df['diff'])]+['']*(len(df)-1)
    df['m_abs'] = [np.mean(df['abs_diff'])]+['']*(len(df)-1)
    
    first = df.iloc[:,0].tolist()
    for i, line in enumerate(first):
        if line <= 28:
            first[i] = 0
    df1 = pd.DataFrame({'contMTC': first})

    frames = [df,df1]
    df = pd.concat(frames, axis=1)
    
    return df

### Add ioi filter and proceed last calculations

In [152]:
#RUN!!
def final_concat_120(file, directory):
    excels = ioi_limits_120(directory)
    
    df1 = cont_mtc_120(file)
    df2 = pd.read_excel('df_ioi_120.xlsx', usecols=lambda x: 'Unnamed' not in x,)
    
    df2 = df2.drop('all_120', axis=1)
    
    frames = [df1, df2]
    df = pd.concat(frames, axis=1)
    
    os.remove("df_ioi_120.xlsx")
    
    return df

## Cont IOI measures

From contMTC delete if difference is out of calculated ioi limits.

In [153]:
#RUN!!
def pre_ioi_filt_120(file, directory):
    df = final_concat_120(file, directory)
    
    df['Difference'] = df['contMTC'].diff()
    first = df['Difference'].tolist()
    
    for i, vrstica in enumerate(first):
        if vrstica < (df['min'].iloc[0]) or vrstica > (df['max'].iloc[0]):
            first[i] = 0            
            
    df1 = pd.DataFrame({'preIOI': first})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)
    df = df.drop('Difference', axis = 1)
    
    col = df['preIOI'].tolist()
    st = 0

    for i, line in enumerate(col):
        if line != 0 and st <= 29:
            st += 1
        elif line != 0 and st > 29:
            col[i] = 0

    df1 = pd.DataFrame({'preIOI_29': col})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)
    
    return df

ContIOI calculated: delete smaller than 0.1 and outliers based on quartiles of participant (preIOI quartiles).

In [154]:
#RUN!!!

def cont_ioi_final_120(file, directory):
    df = pre_ioi_filt_120(file, directory)
    df['contMTC'].replace(to_replace = 0, value = float('NaN'), inplace=True)
    df['preIOI'].replace(to_replace = 0, value = float('NaN'), inplace=True)
    df['preIOI_29'].replace(to_replace = 0, value = float('NaN'), inplace=True)
    
    first = df['preIOI_29'].tolist()
    
    q1 = np.quantile(first, .25)
    q3 = np.quantile(first, .75)
    iqr = q3 - q1
    
    for i, vrstica in enumerate(first):
        if vrstica < 0.1 or vrstica < (q1 - (3*iqr)) or vrstica > (q3 + (3*iqr)):
            first[i] = 0
            
            
    df1 = pd.DataFrame({'contIOI': first})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)
    df['contIOI'].replace(to_replace = 0, value = float('NaN'), inplace=True)
    
    return df

Last measures: avg, std, IOI elemszám, TÖRÖLNI.

In [155]:
#RUN!!
#Change get_name in final version!

def all_calc_120(file, directory):
    df = cont_ioi_final_120(file, directory)
    #name = get_name1(file)
    
    df['mIOIcont'] = [df['contIOI'].mean()]+['']*(len(df)-1)
    df['CVcont'] = [(df['contIOI'].std())/(df['mIOIcont'].iloc[0])]+['']*(len(df)-1)
    
    df['IOIelemszam'] = [((df['contIOI'] != 0).sum())]+['']*(len(df)-1)
    
    return df

# 120m - Functions on one sheet

Everything same as in 120, just on metronome, all calculations are same.

### IOI limits calculation

##### Output just ioi data - for concatenate in IOI

In [156]:
#RUN
def ioi_lim_alone_120m(file):
    df = read_edit_csv3(file)
    name = get_name2(file)
    
    df['Difference'] = df.diff()
    first = df.iloc[:,0].tolist()
    second = df.iloc[:,1].tolist()

    for vrstica1 in first:
        for i, vrstica2 in enumerate(second):
            if vrstica2 == vrstica1 or vrstica2 < 0.1:
                vrstica2 = float('NaN')
                second[i] = vrstica2
            
    df1 = pd.DataFrame({'ioi':second})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)
    df = df.drop('Difference', axis = 1)
    df = df.rename(columns={"ioi": name})
    df = df.iloc[: , 1:]
    return df

Add to raw data clean (1.75<x<28)

In [157]:
#RUN!
def ioi_lim_clean_120m(file):
    df = read_edit_csv3(file)
    
    first = df.iloc[:,0].tolist()
    for i, line in enumerate(first):
        if line < 1.7 or line > 28:
            first[i] = 0
    df1 = pd.DataFrame({'clean': first})
    
    frames = [df,df1]
    df = pd.concat(frames, axis=1)

    return df

#### Outlier filtering 

##### Delete first 10 valid taps

After filtering, the first 10 valid taps are deleted from the beginning of each trial)

In [158]:
#RUN!
def ioi_lim_filt10_120m(file):
    df = ioi_lim_clean_120m(file)
    
    df['diff'] = df['clean'].diff()
    first = df.iloc[:,1].tolist()
    second = df.iloc[:,2].tolist()
    
    delete = 0
    for vrstica1 in first:
        for i, vrstica2 in enumerate(second):
            if vrstica2 == vrstica1 or vrstica2 < 0.1:
                second[i] = 0
            elif vrstica2 != 0 and delete < 10:
                second[i] = 0
                delete += 1
    
                           
    df1 = pd.DataFrame({'filter10':second})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)

    return df

Function for clean, filter and delete first 10 valid taps

In [159]:
#RUN!!
def ioi_lim_clean10_120m(file):
    df = ioi_lim_filt10_120m(file)
       
    
    df['filtered'] = np.where(df['filter10'] == 0,
                             df['filter10'], df['clean'])
    
    df['filtered'].replace(to_replace = 0, value = float('NaN'), inplace=True)

    return df #return data frame with columns: input track, clean, diff, filter10, filtered

### Calculations

Add REF column to the data frame

In [182]:
#RUN!!
def df_prep_120m(file):
    df = ioi_lim_clean10_120m(file)
    df = df.drop(['clean', 'diff', 'filter10'], axis=1)
    
    s = [0]
    x = 0
    while x < 28:
        x += 0.5
        s.append(x)

    df['REF'] = pd.Series(s)

    return df

Diff, nearest values

In [161]:
#RUN!!
def df_diff_angle_120m(file):
    df = df_prep_120m(file)
    
    df['diff'] = (pd.merge_asof(df['filtered'].dropna().sort_values().reset_index(),
                                df[['REF']].dropna().sort_values('REF'),
                                left_on='filtered', right_on='REF', direction='nearest')
                    .set_index('index')['REF'].rsub(df['filtered'])
                  )
    
    df['angle'] = (df['diff']/0.5)*360
    
    return df

### IOI data

Give you data frame with ioi data from all participants.

In [162]:
#RUN!
def get_files_120m(directory):
    seznam = []

    for dirpath, dirnames, filenames in os.walk(directory): #directory with all folders of participants
        for filename in [f for f in filenames if f.endswith("120m.csv")]: #search for all 120m files
            seznam.append(os.path.join(dirpath, filename))
            
    return seznam

In [163]:
#RUN!!
def ioi_frame_120m(directory):
    
    seznam = get_files_120m(directory)
    li = []    
    
    for filename in seznam:
        df = ioi_lim_alone_120m(filename)
        li.append(df)

    frame = pd.concat(li, axis=1, ignore_index=False)
    
    return frame

##### Concatenate just IOI data

### IOI-limits

Add columns with all values together and calculate M, SD, Q1, Q3, IQR, MAX, MIN.

In [164]:
#RUN!
def list_ioi_calc_120m(directory):
    frame = ioi_frame_120m(directory)
    frame['combined']= frame.values.tolist()
    li = []

    for row in frame['combined']:
        for vsak in row:
            li.append(vsak)

    new_array = [x for x in li if math.isnan(x) == False]
    se = pd.Series(new_array)
    
    df_ioi = pd.DataFrame([new_array])
    df_ioi = df_ioi.T
    df_ioi['all_120m'] = pd.DataFrame(data=se)
    df_ioi = df_ioi.iloc[: , 1:]
    
    return df_ioi

In [165]:
#RUN!!
def ioi_limits_120m(directory):
    frame = ioi_frame_120m(directory)
    df_ioi = list_ioi_calc_120m(directory)
    
    df_ioi['mean'] = [(df_ioi['all_120m'].mean())]+['']*(len(df_ioi)-1)
    frame['mean'] = df_ioi['mean']
    
    df_ioi['std'] = [(df_ioi['all_120m'].std())]+['']*(len(df_ioi)-1)
    frame['std'] = df_ioi['std']
    
    df_ioi['q1'] = [(df_ioi['all_120m'].quantile(.25, interpolation="nearest"))]+['']*(len(df_ioi)-1)
    frame['q1'] = df_ioi['q1']
    
    df_ioi['q3'] = [(df_ioi['all_120m'].quantile(.75, interpolation="nearest"))]+['']*(len(df_ioi)-1)
    frame['q3'] = df_ioi['q3']
    
    df_ioi['IQR'] = [(df_ioi['q3'].iloc[0] - df_ioi['q1'].iloc[0])]+['']*(len(df_ioi)-1)
    frame['IQR'] = df_ioi['IQR']
    
    df_ioi['min'] = [(df_ioi['q1'].iloc[0] - (3*(df_ioi['IQR'].iloc[0])))]+['']*(len(df_ioi)-1)
    frame['min'] = df_ioi['min']
    
    df_ioi['max'] = [(df_ioi['q3'].iloc[0] + (3*(df_ioi['IQR'].iloc[0])))]+['']*(len(df_ioi)-1)
    frame['max'] = df_ioi['max']
    
    return df_ioi.to_excel('df_ioi_120m.xlsx'), frame.to_excel('IOI_filter_120m.xlsx')

### Circular analysis

In [166]:
#RUN!!
def circular_120m(file):
    prep = prep_for_r_120m(file)
    circ_in_r_120m(prep)

Create xlsx with angel column to send in R

In [167]:
#RUN!!
def prep_for_r_120m(file):
    df = df_diff_angle_120m(file)
    name = get_name1(file)

    df_a = df.filter(['angle'])
        
    return df_a.to_excel('angle_120m.xlsx')

Call R function and provide calculations, merge them in participant dataframe

In [168]:
#RUN!!

#Excecute R circular analysis and make xlsx file with measurement

def circ_in_r_120m(prep):

    robjects.r('''
        circ_anal <- function (xlsx_file) {
          library(readxl)
          library(circular)
          library(openxlsx)
          library(readr)
          library(rlang)

          angle_120m <- read_excel(xlsx_file)

          angle_120m[1] <- NULL

          circ120m <- data.frame(ID = colnames(angle_120m), 
                               RAY = NA,
                               RAYsig = NA,
                               MEAN = NA,
                               Rho = NA)

          for(i in 1:length(circ120m$ID)){
            a <- angle_120m[,i]
            a_circ <- circular(a, units = "degrees", zero = circular(pi/2), rotation = "clock")
            circ120m$RAY[i] <- rayleigh.test(a_circ)$statistic
            circ120m$RAYsig[i] <- rayleigh.test(a_circ)$p.value
            circ120m$MEAN[i] <- mean.circular(a_circ, na.rm = TRUE)
            circ120m$Rho[i] <- rho.circular(a_circ, na.rm = TRUE)

          }


          write.xlsx(circ120m,"C:/Users/Neza/Downloads/tapping/circ120m_stat.xlsx")

        }

        circ_anal("C:/Users/Neza/Downloads/tapping/angle_120m.xlsx")
    ''')

Read excel with circular data and delete angle_80 and circ80_stat files.

In [169]:
#RUN!!!

def final_circ_120m(file):
    circular_120m(file)
    
    df1 = df_diff_angle_120m(file)
    df2 = pd.read_excel('circ120m_stat.xlsx')

    df = pd.concat([df1, df2], axis = 1)
    df = df.drop('ID', axis=1)

    os.remove("angle_120m.xlsx")
    os.remove("circ120m_stat.xlsx")
    
    return df

## Cont MTC

Add abs(DIF) column, calculated both means, add contMTC column.

In [170]:
#RUN!!
def cont_mtc_120m(file):
    df = final_circ_120m(file)
    
    df['abs_diff'] = df['diff'].abs()
    df['m_diff'] = [np.mean(df['diff'])]+['']*(len(df)-1)
    df['m_abs'] = [np.mean(df['abs_diff'])]+['']*(len(df)-1)
    
    first = df.iloc[:,0].tolist()
    for i, line in enumerate(first):
        if line <= 28:
            first[i] = 0
    df1 = pd.DataFrame({'contMTC': first})

    frames = [df,df1]
    df = pd.concat(frames, axis=1)
    
    return df

### Add ioi filter and proceed last calculations

In [219]:
#RUN!!
def final_concat_120m(file, directory):
    excels = ioi_limits_120m(directory)
    
    df1 = cont_mtc_120m(file)
    df2 = pd.read_excel('df_ioi_120m.xlsx', usecols=lambda x: 'Unnamed' not in x,)
    
    df2 = df2.drop('all_120m', axis=1)
    
    frames = [df1, df2]
    df = pd.concat(frames, axis=1)
    
    os.remove("df_ioi_120m.xlsx")
    
    return df

## Cont IOI measures

From contMTC delete if difference is out of calculated ioi limits.

In [172]:
#RUN!!
def pre_ioi_filt_120m(file, directory):
    df = final_concat_120m(file, directory)
    
    df['Difference'] = df['contMTC'].diff()
    first = df['Difference'].tolist()
    
    for i, vrstica in enumerate(first):
        if vrstica < (df['min'].iloc[0]) or vrstica > (df['max'].iloc[0]):
            first[i] = 0            
            
    df1 = pd.DataFrame({'preIOI': first})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)
    df = df.drop('Difference', axis = 1)
    
    col = df['preIOI'].tolist()
    st = 0

    for i, line in enumerate(col):
        if line != 0 and st <= 29:
            st += 1
        elif line != 0 and st > 29:
            col[i] = 0

    df1 = pd.DataFrame({'preIOI_29': col})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)
    
    return df

ContIOI calculated: delete smaller than 0.1 and outliers based on quartiles of participant (preIOI quartiles).

In [173]:
#RUN!!!

def cont_ioi_final_120m(file, directory):
    df = pre_ioi_filt_120m(file, directory)
    df['contMTC'].replace(to_replace = 0, value = float('NaN'), inplace=True)
    df['preIOI'].replace(to_replace = 0, value = float('NaN'), inplace=True)
    df['preIOI_29'].replace(to_replace = 0, value = float('NaN'), inplace=True)
    
    first = df['preIOI_29'].tolist()
    
    q1 = np.quantile(first, .25)
    q3 = np.quantile(first, .75)
    iqr = q3 - q1
    
    for i, vrstica in enumerate(first):
        if vrstica < 0.1 or vrstica < (q1 - (3*iqr)) or vrstica > (q3 + (3*iqr)):
            first[i] = 0
            
            
    df1 = pd.DataFrame({'contIOI': first})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)
    df['contIOI'].replace(to_replace = 0, value = float('NaN'), inplace=True)
    
    return df

Last measures: avg, std, IOI elemszám, TÖRÖLNI.

In [174]:
#RUN!!
#Change get_name in final version!

def all_calc_120m(file, directory):
    df = cont_ioi_final_120m(file, directory)
    name = get_name1(file)
    
    df['mIOIcont'] = [df['contIOI'].mean()]+['']*(len(df)-1)
    df['CVcont'] = [(df['contIOI'].std())/(df['mIOIcont'].iloc[0])]+['']*(len(df)-1)
    
    df['IOIelemszam'] = [((df['contIOI'] != 0).sum())]+['']*(len(df)-1)
    #df['torolni'] = [(df['IOIelemszam'].iloc[0]) - 29]+['']*(len(df)-1)
    
    return df

# 150

### IOI limits calculation

##### Output just ioi data - for concatenate in IOI

In [184]:
#RUN
def ioi_lim_alone_150(file):
    df = read_edit_csv3(file)
    name = get_name1(file)
    
    df['Difference'] = df.diff()
    first = df.iloc[:,0].tolist()
    second = df.iloc[:,1].tolist()

    for vrstica1 in first:
        for i, vrstica2 in enumerate(second):
            if vrstica2 == vrstica1 or vrstica2 < 0.1:
                vrstica2 = float('NaN')
                second[i] = vrstica2
            
    df1 = pd.DataFrame({'ioi':second})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)
    df = df.drop('Difference', axis = 1)
    df = df.rename(columns={"ioi": name})
    df = df.iloc[: , 1:]
    return df

Add to raw data clean (1.4<x<20.8)

In [185]:
#RUN!
def ioi_lim_clean_150(file):
    df = read_edit_csv3(file)
    
    first = df.iloc[:,0].tolist()
    for i, line in enumerate(first):
        if line < 1.4 or line > 20.8:
            first[i] = 0
    df1 = pd.DataFrame({'clean': first})
    
    frames = [df,df1]
    df = pd.concat(frames, axis=1)

    return df

#### Outlier filtering 

##### Delete first 10 valid taps

After filtering, the first 10 valid taps are deleted from the beginning of each trial)

In [186]:
#RUN!
def ioi_lim_filt10_150(file):
    df = ioi_lim_clean_150(file)
    
    df['diff'] = df['clean'].diff()
    first = df.iloc[:,1].tolist()
    second = df.iloc[:,2].tolist()
    
    delete = 0
    for vrstica1 in first:
        for i, vrstica2 in enumerate(second):
            if vrstica2 == vrstica1 or vrstica2 < 0.1:
                second[i] = 0
            elif vrstica2 != 0 and delete < 10:
                second[i] = 0
                delete += 1
    
                           
    df1 = pd.DataFrame({'filter10':second})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)

    return df

Function for clean, filter and delete first 10 valid taps

In [187]:
#RUN!!
def ioi_lim_clean10_150(file):
    df = ioi_lim_filt10_150(file)
       
    
    df['filtered'] = np.where(df['filter10'] == 0,
                             df['filter10'], df['clean'])
    
    df['filtered'].replace(to_replace = 0, value = float('NaN'), inplace=True)

    return df #return data frame with columns: input track, clean, diff, filter10, filtered

### Calculations

Add REF column to the data frame

In [226]:
#RUN!!
def df_prep_150(file):
    df = ioi_lim_clean10_150(file)
    df = df.drop(['clean', 'diff', 'filter10'], axis=1)
    
    s = [0]
    x = 0
    while x < 20.4:
        x += 0.4
        s.append(x)

    df['REF'] = pd.Series(s)

    return df

Diff, nearest values

In [189]:
#RUN!!
def df_diff_angle_150(file):
    df = df_prep_150(file)
    
    df['diff'] = (pd.merge_asof(df['filtered'].dropna().sort_values().reset_index(),
                                df[['REF']].dropna().sort_values('REF'),
                                left_on='filtered', right_on='REF', direction='nearest')
                    .set_index('index')['REF'].rsub(df['filtered'])
                  )
    
    df['angle'] = (df['diff']/0.4)*360
    
    return df

### IOI data

Give you data frame with ioi data from all participants.

In [190]:
#RUN!
def get_files_150(directory):
    seznam = []

    for dirpath, dirnames, filenames in os.walk(directory): #directory with all folders of participants
        for filename in [f for f in filenames if f.endswith("150.csv")]: #search for all 80 files
            seznam.append(os.path.join(dirpath, filename))
            
    return seznam

In [191]:
#RUN!!
def ioi_frame_150(directory):
    
    seznam = get_files_150(directory)
    li = []    
    
    for filename in seznam:
        df = ioi_lim_alone_150(filename)
        li.append(df)

    frame = pd.concat(li, axis=1, ignore_index=False)
    
    return frame

##### Concatenate just IOI data

### IOI-limits

Add columns with all values together and calculate M, SD, Q1, Q3, IQR, MAX, MIN.

In [192]:
#RUN!
def list_ioi_calc_150(directory):
    frame = ioi_frame_150(directory)
    frame['combined']= frame.values.tolist()
    li = []

    for row in frame['combined']:
        for vsak in row:
            li.append(vsak)

    new_array = [x for x in li if math.isnan(x) == False]
    se = pd.Series(new_array)
    
    df_ioi = pd.DataFrame([new_array])
    df_ioi = df_ioi.T
    df_ioi['all_150'] = pd.DataFrame(data=se)
    df_ioi = df_ioi.iloc[: , 1:]
    
    return df_ioi

In [193]:
#RUN!!
def ioi_limits_150(directory):
    frame = ioi_frame_150(directory)
    df_ioi = list_ioi_calc_150(directory)
    
    df_ioi['mean'] = [(df_ioi['all_150'].mean())]+['']*(len(df_ioi)-1)
    frame['mean'] = df_ioi['mean']
    
    df_ioi['std'] = [(df_ioi['all_150'].std())]+['']*(len(df_ioi)-1)
    frame['std'] = df_ioi['std']
    
    df_ioi['q1'] = [(df_ioi['all_150'].quantile(.25, interpolation="nearest"))]+['']*(len(df_ioi)-1)
    frame['q1'] = df_ioi['q1']
    
    df_ioi['q3'] = [(df_ioi['all_150'].quantile(.75, interpolation="nearest"))]+['']*(len(df_ioi)-1)
    frame['q3'] = df_ioi['q3']
    
    df_ioi['IQR'] = [(df_ioi['q3'].iloc[0] - df_ioi['q1'].iloc[0])]+['']*(len(df_ioi)-1)
    frame['IQR'] = df_ioi['IQR']
    
    df_ioi['min'] = [(df_ioi['q1'].iloc[0] - (3*(df_ioi['IQR'].iloc[0])))]+['']*(len(df_ioi)-1)
    frame['min'] = df_ioi['min']
    
    df_ioi['max'] = [(df_ioi['q3'].iloc[0] + (3*(df_ioi['IQR'].iloc[0])))]+['']*(len(df_ioi)-1)
    frame['max'] = df_ioi['max']
    
    return df_ioi.to_excel('df_ioi_150.xlsx'), frame.to_excel('IOI_filter_150.xlsx')

### Circular analysis

In [194]:
#RUN!!
def circular_150(file):
    prep = prep_for_r_150(file)
    circ_in_r_150(prep)

Create xlsx with angel column to send in R

In [195]:
#RUN!!
def prep_for_r_150(file):
    df = df_diff_angle_150(file)
    name = get_name1(file)

    df_a = df.filter(['angle'])
        
    return df_a.to_excel('angle_150.xlsx')

Call R function and provide calculations, merge them in participant dataframe

In [196]:
#RUN!!

#Excecute R circular analysis and make xlsx file with measurement
#Change to right path, where should be excel files made

def circ_in_r_150(prep):

    robjects.r('''
        circ_anal <- function (xlsx_file) {
          library(readxl)
          library(circular)
          library(openxlsx)
          library(readr)
          library(rlang)

          angle_150 <- read_excel(xlsx_file)

          angle_150[1] <- NULL

          circ150 <- data.frame(ID = colnames(angle_150), 
                               RAY = NA,
                               RAYsig = NA,
                               MEAN = NA,
                               Rho = NA)

          for(i in 1:length(circ150$ID)){
            a <- angle_150[,i]
            a_circ <- circular(a, units = "degrees", zero = circular(pi/2), rotation = "clock")
            circ150$RAY[i] <- rayleigh.test(a_circ)$statistic
            circ150$RAYsig[i] <- rayleigh.test(a_circ)$p.value
            circ150$MEAN[i] <- mean.circular(a_circ, na.rm = TRUE)
            circ150$Rho[i] <- rho.circular(a_circ, na.rm = TRUE)

          }


          write.xlsx(circ150,"C:/Users/Neza/Downloads/tapping/circ150_stat.xlsx")

        }

        circ_anal("C:/Users/Neza/Downloads/tapping/angle_150.xlsx")
    ''')

Read excel with circular data and delete angle_80 and circ80_stat files.

In [197]:
#RUN!!!

def final_circ_150(file):
    circular_150(file)
    
    df1 = df_diff_angle_150(file)
    df2 = pd.read_excel('circ150_stat.xlsx')

    df = pd.concat([df1, df2], axis = 1)
    df = df.drop('ID', axis=1)

    os.remove("angle_150.xlsx")
    os.remove("circ150_stat.xlsx")
    
    return df

## Cont MTC

Add abs(DIF) column, calculated both means, add contMTC column.

In [198]:
#RUN!!
def cont_mtc_150(file):
    df = final_circ_150(file)
    
    df['abs_diff'] = df['diff'].abs()
    df['m_diff'] = [np.mean(df['diff'])]+['']*(len(df)-1)
    df['m_abs'] = [np.mean(df['abs_diff'])]+['']*(len(df)-1)
    
    first = df.iloc[:,0].tolist()
    for i, line in enumerate(first):
        if line <= 20.8:
            first[i] = 0
    df1 = pd.DataFrame({'contMTC': first})

    frames = [df,df1]
    df = pd.concat(frames, axis=1)
    
    return df

### Add ioi filter and proceed last calculations

In [199]:
#RUN!!
def final_concat_150(file, directory):
    excels = ioi_limits_150(directory)
    
    df1 = cont_mtc_150(file)
    df2 = pd.read_excel('df_ioi_150.xlsx', usecols=lambda x: 'Unnamed' not in x,)
    
    df2 = df2.drop('all_150', axis=1)
    
    frames = [df1, df2]
    df = pd.concat(frames, axis=1)
    
    os.remove("df_ioi_150.xlsx")
    
    return df

## Cont IOI measures

From contMTC delete if difference is out of calculated ioi limits.

In [200]:
#RUN!!
def pre_ioi_filt_150(file, directory):
    df = final_concat_150(file, directory)
    
    df['Difference'] = df['contMTC'].diff()
    first = df['Difference'].tolist()
    
    for i, vrstica in enumerate(first):
        if vrstica < (df['min'].iloc[0]) or vrstica > (df['max'].iloc[0]):
            first[i] = 0            
            
    df1 = pd.DataFrame({'preIOI': first})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)
    df = df.drop('Difference', axis = 1)
    
    col = df['preIOI'].tolist()
    st = 0

    for i, line in enumerate(col):
        if line != 0 and st <= 29:
            st += 1
        elif line != 0 and st > 29:
            col[i] = 0

    df1 = pd.DataFrame({'preIOI_29': col})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)
    
    return df

ContIOI calculated: delete smaller than 0.1 and outliers based on quartiles of participant (preIOI quartiles).

In [201]:
#RUN!!!

def cont_ioi_final_150(file, directory):
    df = pre_ioi_filt_150(file, directory)
    df['contMTC'].replace(to_replace = 0, value = float('NaN'), inplace=True)
    df['preIOI'].replace(to_replace = 0, value = float('NaN'), inplace=True)
    df['preIOI_29'].replace(to_replace = 0, value = float('NaN'), inplace=True)
    
    first = df['preIOI_29'].tolist()
    
    q1 = np.quantile(first, .25)
    q3 = np.quantile(first, .75)
    iqr = q3 - q1
    
    for i, vrstica in enumerate(first):
        if vrstica < 0.1 or vrstica < (q1 - (3*iqr)) or vrstica > (q3 + (3*iqr)):
            first[i] = 0
            
            
    df1 = pd.DataFrame({'contIOI': first})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)
    df['contIOI'].replace(to_replace = 0, value = float('NaN'), inplace=True)
    
    return df

Last measures: avg, std, IOI elemszám, TÖRÖLNI.

In [202]:
#RUN!!
#Change get_name in final version!

def all_calc_150(file, directory):
    df = cont_ioi_final_150(file, directory)
    #name = get_name1(file)
    
    df['mIOIcont'] = [df['contIOI'].mean()]+['']*(len(df)-1)
    df['CVcont'] = [(df['contIOI'].std())/(df['mIOIcont'].iloc[0])]+['']*(len(df)-1)
    
    df['IOIelemszam'] = [((df['contIOI'] != 0).sum())]+['']*(len(df)-1)
    
    return df

# 150m - Functions on one sheet

Everything same as in 150, just on metronome, all calculations are same.

### IOI limits calculation

##### Output just ioi data - for concatenate in IOI

In [203]:
#RUN
def ioi_lim_alone_150m(file):
    df = read_edit_csv3(file)
    name = get_name2(file)
    
    df['Difference'] = df.diff()
    first = df.iloc[:,0].tolist()
    second = df.iloc[:,1].tolist()

    for vrstica1 in first:
        for i, vrstica2 in enumerate(second):
            if vrstica2 == vrstica1 or vrstica2 < 0.1:
                vrstica2 = float('NaN')
                second[i] = vrstica2
            
    df1 = pd.DataFrame({'ioi':second})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)
    df = df.drop('Difference', axis = 1)
    df = df.rename(columns={"ioi": name})
    df = df.iloc[: , 1:]
    return df

Add to raw data clean (1.4<x<20.8)

In [204]:
#RUN!
def ioi_lim_clean_150m(file):
    df = read_edit_csv3(file)
    
    first = df.iloc[:,0].tolist()
    for i, line in enumerate(first):
        if line < 1.4 or line > 20.8:
            first[i] = 0
    df1 = pd.DataFrame({'clean': first})
    
    frames = [df,df1]
    df = pd.concat(frames, axis=1)

    return df

#### Outlier filtering 

##### Delete first 10 valid taps

After filtering, the first 10 valid taps are deleted from the beginning of each trial)

In [205]:
#RUN!
def ioi_lim_filt10_150m(file):
    df = ioi_lim_clean_150m(file)
    
    df['diff'] = df['clean'].diff()
    first = df.iloc[:,1].tolist()
    second = df.iloc[:,2].tolist()
    
    delete = 0
    for vrstica1 in first:
        for i, vrstica2 in enumerate(second):
            if vrstica2 == vrstica1 or vrstica2 < 0.1:
                second[i] = 0
            elif vrstica2 != 0 and delete < 10:
                second[i] = 0
                delete += 1
    
                           
    df1 = pd.DataFrame({'filter10':second})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)

    return df

Function for clean, filter and delete first 10 valid taps

In [206]:
#RUN!!
def ioi_lim_clean10_150m(file):
    df = ioi_lim_filt10_150m(file)
       
    
    df['filtered'] = np.where(df['filter10'] == 0,
                             df['filter10'], df['clean'])
    
    df['filtered'].replace(to_replace = 0, value = float('NaN'), inplace=True)

    return df #return data frame with columns: input track, clean, diff, filter10, filtered

### Calculations

Add REF column to the data frame

In [225]:
#RUN!!
def df_prep_150m(file):
    df = ioi_lim_clean10_150m(file)
    df = df.drop(['clean', 'diff', 'filter10'], axis=1)
    
    s = [0]
    x = 0
    while x < 20.4:
        x += 0.4
        s.append(x)

    df['REF'] = pd.Series(s)

    return df

Diff, nearest values

In [208]:
#RUN!!
def df_diff_angle_150m(file):
    df = df_prep_150m(file)
    
    df['diff'] = (pd.merge_asof(df['filtered'].dropna().sort_values().reset_index(),
                                df[['REF']].dropna().sort_values('REF'),
                                left_on='filtered', right_on='REF', direction='nearest')
                    .set_index('index')['REF'].rsub(df['filtered'])
                  )
    
    df['angle'] = (df['diff']/0.4)*360
    
    return df

### IOI data

Give you data frame with ioi data from all participants.

In [209]:
#RUN!
def get_files_150m(directory):
    seznam = []

    for dirpath, dirnames, filenames in os.walk(directory): #directory with all folders of participants
        for filename in [f for f in filenames if f.endswith("150m.csv")]: #search for all 150m files
            seznam.append(os.path.join(dirpath, filename))
            
    return seznam

In [210]:
#RUN!!
def ioi_frame_150m(directory):
    
    seznam = get_files_150m(directory)
    li = []    
    
    for filename in seznam:
        df = ioi_lim_alone_150m(filename)
        li.append(df)

    frame = pd.concat(li, axis=1, ignore_index=False)
    
    return frame

##### Concatenate just IOI data

### IOI-limits

Add columns with all values together and calculate M, SD, Q1, Q3, IQR, MAX, MIN.

In [211]:
#RUN!
def list_ioi_calc_150m(directory):
    frame = ioi_frame_150m(directory)
    frame['combined']= frame.values.tolist()
    li = []

    for row in frame['combined']:
        for vsak in row:
            li.append(vsak)

    new_array = [x for x in li if math.isnan(x) == False]
    se = pd.Series(new_array)
    
    df_ioi = pd.DataFrame([new_array])
    df_ioi = df_ioi.T
    df_ioi['all_150m'] = pd.DataFrame(data=se)
    df_ioi = df_ioi.iloc[: , 1:]
    
    return df_ioi

In [212]:
#RUN!!
def ioi_limits_150m(directory):
    frame = ioi_frame_150m(directory)
    df_ioi = list_ioi_calc_150m(directory)
    
    df_ioi['mean'] = [(df_ioi['all_150m'].mean())]+['']*(len(df_ioi)-1)
    frame['mean'] = df_ioi['mean']
    
    df_ioi['std'] = [(df_ioi['all_150m'].std())]+['']*(len(df_ioi)-1)
    frame['std'] = df_ioi['std']
    
    df_ioi['q1'] = [(df_ioi['all_150m'].quantile(.25, interpolation="nearest"))]+['']*(len(df_ioi)-1)
    frame['q1'] = df_ioi['q1']
    
    df_ioi['q3'] = [(df_ioi['all_150m'].quantile(.75, interpolation="nearest"))]+['']*(len(df_ioi)-1)
    frame['q3'] = df_ioi['q3']
    
    df_ioi['IQR'] = [(df_ioi['q3'].iloc[0] - df_ioi['q1'].iloc[0])]+['']*(len(df_ioi)-1)
    frame['IQR'] = df_ioi['IQR']
    
    df_ioi['min'] = [(df_ioi['q1'].iloc[0] - (3*(df_ioi['IQR'].iloc[0])))]+['']*(len(df_ioi)-1)
    frame['min'] = df_ioi['min']
    
    df_ioi['max'] = [(df_ioi['q3'].iloc[0] + (3*(df_ioi['IQR'].iloc[0])))]+['']*(len(df_ioi)-1)
    frame['max'] = df_ioi['max']
    
    return df_ioi.to_excel('df_ioi_150m.xlsx'), frame.to_excel('IOI_filter_150m.xlsx')

### Circular analysis

In [213]:
#RUN!!
def circular_150m(file):
    prep = prep_for_r_150m(file)
    circ_in_r_150m(prep)

Create xlsx with angel column to send in R

In [214]:
#RUN!!
def prep_for_r_150m(file):
    df = df_diff_angle_150m(file)
    name = get_name1(file)

    df_a = df.filter(['angle'])
        
    return df_a.to_excel('angle_150m.xlsx')

Call R function and provide calculations, merge them in participant dataframe

In [215]:
#RUN!!

#Excecute R circular analysis and make xlsx file with measurement

def circ_in_r_150m(prep):

    robjects.r('''
        circ_anal <- function (xlsx_file) {
          library(readxl)
          library(circular)
          library(openxlsx)
          library(readr)
          library(rlang)

          angle_150m <- read_excel(xlsx_file)

          angle_150m[1] <- NULL

          circ150m <- data.frame(ID = colnames(angle_150m), 
                               RAY = NA,
                               RAYsig = NA,
                               MEAN = NA,
                               Rho = NA)

          for(i in 1:length(circ150m$ID)){
            a <- angle_150m[,i]
            a_circ <- circular(a, units = "degrees", zero = circular(pi/2), rotation = "clock")
            circ150m$RAY[i] <- rayleigh.test(a_circ)$statistic
            circ150m$RAYsig[i] <- rayleigh.test(a_circ)$p.value
            circ150m$MEAN[i] <- mean.circular(a_circ, na.rm = TRUE)
            circ150m$Rho[i] <- rho.circular(a_circ, na.rm = TRUE)

          }


          write.xlsx(circ150m,"C:/Users/Neza/Downloads/tapping/circ150m_stat.xlsx")

        }

        circ_anal("C:/Users/Neza/Downloads/tapping/angle_150m.xlsx")
    ''')

Read excel with circular data and delete angle_80 and circ80_stat files.

In [216]:
#RUN!!!

def final_circ_150m(file):
    circular_150m(file)
    
    df1 = df_diff_angle_150m(file)
    df2 = pd.read_excel('circ150m_stat.xlsx')

    df = pd.concat([df1, df2], axis = 1)
    df = df.drop('ID', axis=1)

    os.remove("angle_150m.xlsx")
    os.remove("circ150m_stat.xlsx")
    
    return df

## Cont MTC

Add abs(DIF) column, calculated both means, add contMTC column.

In [217]:
#RUN!!
def cont_mtc_150m(file):
    df = final_circ_150m(file)
    
    df['abs_diff'] = df['diff'].abs()
    df['m_diff'] = [np.mean(df['diff'])]+['']*(len(df)-1)
    df['m_abs'] = [np.mean(df['abs_diff'])]+['']*(len(df)-1)
    
    first = df.iloc[:,0].tolist()
    for i, line in enumerate(first):
        if line <= 20.8:
            first[i] = 0
    df1 = pd.DataFrame({'contMTC': first})

    frames = [df,df1]
    df = pd.concat(frames, axis=1)
    
    return df

### Add ioi filter and proceed last calculations

In [218]:
#RUN!!
def final_concat_150m(file, directory):
    excels = ioi_limits_150m(directory)
    
    df1 = cont_mtc_150m(file)
    df2 = pd.read_excel('df_ioi_150m.xlsx', usecols=lambda x: 'Unnamed' not in x,)
    
    df2 = df2.drop('all_150m', axis=1)
    
    frames = [df1, df2]
    df = pd.concat(frames, axis=1)
    
    os.remove("df_ioi_150m.xlsx")
    
    return df

## Cont IOI measures

From contMTC delete if difference is out of calculated ioi limits.

In [220]:
#RUN!!
def pre_ioi_filt_150m(file, directory):
    df = final_concat_150m(file, directory)
    
    df['Difference'] = df['contMTC'].diff()
    first = df['Difference'].tolist()
    
    for i, vrstica in enumerate(first):
        if vrstica < (df['min'].iloc[0]) or vrstica > (df['max'].iloc[0]):
            first[i] = 0            
            
    df1 = pd.DataFrame({'preIOI': first})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)
    df = df.drop('Difference', axis = 1)
    
    col = df['preIOI'].tolist()
    st = 0

    for i, line in enumerate(col):
        if line != 0 and st <= 29:
            st += 1
        elif line != 0 and st > 29:
            col[i] = 0

    df1 = pd.DataFrame({'preIOI_29': col})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)
    
    return df

ContIOI calculated: delete smaller than 0.1 and outliers based on quartiles of participant (preIOI quartiles).

In [221]:
#RUN!!!

def cont_ioi_final_150m(file, directory):
    df = pre_ioi_filt_150m(file, directory)
    df['contMTC'].replace(to_replace = 0, value = float('NaN'), inplace=True)
    df['preIOI'].replace(to_replace = 0, value = float('NaN'), inplace=True)
    df['preIOI_29'].replace(to_replace = 0, value = float('NaN'), inplace=True)
    
    first = df['preIOI_29'].tolist()
    
    q1 = np.quantile(first, .25)
    q3 = np.quantile(first, .75)
    iqr = q3 - q1
    
    for i, vrstica in enumerate(first):
        if vrstica < 0.1 or vrstica < (q1 - (3*iqr)) or vrstica > (q3 + (3*iqr)):
            first[i] = 0
            
            
    df1 = pd.DataFrame({'contIOI': first})
    frames = [df,df1]

    df = pd.concat(frames, axis=1)
    df['contIOI'].replace(to_replace = 0, value = float('NaN'), inplace=True)
    
    return df

Last measures: avg, std, IOI elemszám, TÖRÖLNI.

In [222]:
#RUN!!
#Change get_name in final version!

def all_calc_150m(file, directory):
    df = cont_ioi_final_150m(file, directory)
    name = get_name1(file)
    
    df['mIOIcont'] = [df['contIOI'].mean()]+['']*(len(df)-1)
    df['CVcont'] = [(df['contIOI'].std())/(df['mIOIcont'].iloc[0])]+['']*(len(df)-1)
    
    df['IOIelemszam'] = [((df['contIOI'] != 0).sum())]+['']*(len(df)-1)
    #df['torolni'] = [(df['IOIelemszam'].iloc[0]) - 29]+['']*(len(df)-1)
    
    return df

# Merged dataframes - reading through folders

Make all measurements for one participant

Goes through folder with participants folders and output the list with all correct files from participants.

### Final code

In [223]:
#RUN!!!
def final(directory):

    seznam80 = []
    seznam80m = []
    seznam120 = []
    seznam120m = []
    seznam150 = []
    seznam150m = []
    dirp = []

    for dirpath, dirnames, filenames in os.walk(directory): #directory with all folders of participants
            for filename in [f for f in filenames if f.endswith("80.csv")]: #search for all 80 files
                seznam80.append(os.path.join(dirpath, filename))
                dirp.append(dirpath)

    for dirpath, dirnames, filenames in os.walk(directory): #directory with all folders of participants
            for filename in [f for f in filenames if f.endswith("80m.csv")]: #search for all 80m files
                seznam80m.append(os.path.join(dirpath, filename))
    
    for dirpath, dirnames, filenames in os.walk(directory): #directory with all folders of participants
            for filename in [f for f in filenames if f.endswith("120.csv")]: #search for all 120 files
                seznam120.append(os.path.join(dirpath, filename))
    
    for dirpath, dirnames, filenames in os.walk(directory): #directory with all folders of participants
            for filename in [f for f in filenames if f.endswith("120m.csv")]: #search for all 120m files
                seznam120m.append(os.path.join(dirpath, filename))
                
    for dirpath, dirnames, filenames in os.walk(directory): #directory with all folders of participants
            for filename in [f for f in filenames if f.endswith("150.csv")]: #search for all 150 files
                seznam150.append(os.path.join(dirpath, filename))
    
    for dirpath, dirnames, filenames in os.walk(directory): #directory with all folders of participants
            for filename in [f for f in filenames if f.endswith("150m.csv")]: #search for all 150m files
                seznam150m.append(os.path.join(dirpath, filename))


    for vsak80, vsak80m, vsak120, vsak120m, vsak150, vsak150m, pot in zip(seznam80, seznam80m, seznam120, seznam120m, seznam150, seznam150m, dirp):
            path = pot + '_all_anal.xlsx'

            file1 = vsak80
            df1 = all_calc_80(file1, directory)

            file2 = vsak80m
            df2 = all_calc_80m(file2, directory)
            
            file3 = vsak120
            df3 = all_calc_120(file3, directory)

            file4 = vsak120m
            df4 = all_calc_120m(file4, directory)
            
            file5 = vsak150
            df5 = all_calc_150(file5, directory)

            file6 = vsak150m
            df6 = all_calc_150m(file6, directory)

            writer = pd.ExcelWriter(path, engine = 'xlsxwriter')
            df1.to_excel(writer, sheet_name = '80')
            df2.to_excel(writer, sheet_name = '80m')
            df3.to_excel(writer, sheet_name = '120')
            df4.to_excel(writer, sheet_name = '120m')
            df5.to_excel(writer, sheet_name = '150')
            df6.to_excel(writer, sheet_name = '150m')
            writer.close()


# TEST

In [227]:
directory = 'test'
final(directory)

R[write to console]: 
R[write to console]: -
R[write to console]: 
R[write to console]: /
                                                                              
R[write to console]: 
R[write to console]: 
R[write to console]: /
                                                                              
R[write to console]: 
R[write to console]: 
R[write to console]: -
                                                                              
R[write to console]: 


New names:
* `` -> `...1`


R[write to console]: 
R[write to console]: -
R[write to console]: 
R[write to console]: /
                                                                              
R[write to console]: 
R[write to console]: 
R[write to console]: /
                                                                              
R[write to console]: 
R[write to console]: 
R[write to console]: -
                                                                              
R[write to console]: 


New names:
* `` -> `...1`


R[write to console]: 
R[write to console]: -
R[write to console]: 
R[write to console]: /
                                                                              
R[write to console]: 
R[write to console]: 
R[write to console]: /
                                                                              
R[write to console]: 
R[write to console]: 
R[write to console]: -
                                                                              
R[write to console]: 


...

New names:
* `` -> `...1`


R[write to console]: 
R[write to console]: -
R[write to console]: 
R[write to console]: /
                                                                              
R[write to console]: 
R[write to console]: 
R[write to console]: /
                                                                              
R[write to console]: 
R[write to console]: 
R[write to console]: -
                                                                              
R[write to console]: 


New names:
* `` -> `...1`


R[write to console]: 
R[write to console]: -
R[write to console]: 
R[write to console]: /
                                                                              
R[write to console]: 
R[write to console]: 
R[write to console]: /
                                                                              
R[write to console]: 
R[write to console]: 
R[write to console]: -
                                                                              
R[write to console]: 


New names:
* `` -> `...1`


In [ ]:
#For testing, put in function of participant

path = r"test//ab3b//ab3b_all_anal.xlsx"
directory = "test"

file1 = "test//ab3b//ab3b80.csv"
df1 = all_calc_80(file1, directory)

file2 = "test//ab3b//ab3b80m.csv"
df2 = all_calc_80m(file2, directory)

writer = pd.ExcelWriter(path, engine = 'xlsxwriter')
df1.to_excel(writer, sheet_name = '80')
df2.to_excel(writer, sheet_name = '80m')
writer.close()

In [ ]:
file1 = "test//ab3b//ab3b80.csv"
directory = "test"
df1 = all_calc_80(file1, directory1)

In [ ]:
df.head(83)

In [ ]:
directory = 'test'

seznam80 = []
seznam80m = []
dirp = []

for dirpath, dirnames, filenames in os.walk(directory): #directory with all folders of participants
        for filename in [f for f in filenames if f.endswith("80.csv")]: #search for all 80 files
            seznam80.append(os.path.join(dirpath, filename))
            dirp.append(dirpath)
            
for dirpath, dirnames, filenames in os.walk(directory): #directory with all folders of participants
        for filename in [f for f in filenames if f.endswith("80m.csv")]: #search for all 80 files
            seznam80m.append(os.path.join(dirpath, filename))


for vsak1, vsak2, pot in zip(seznam80, seznam80m, dirp):
        path = pot + '_all_anal.xlsx'
        
        file1 = vsak1
        df1 = all_calc_80(file1, directory)
        
        file2 = vsak2
        df2 = all_calc_80m(file2, directory)
        
        writer = pd.ExcelWriter(path, engine = 'xlsxwriter')
        df1.to_excel(writer, sheet_name = '80')
        df2.to_excel(writer, sheet_name = '80m')
        writer.close()
            

